### Turn Cytoscape (v 3.8.2) cluster output into binary fasta

First run Snakefile_final rule __boundary_concat__ to make the "{sample}\_{sample2}\_hc\_conserved\_boundaries" files
Second concatenate (cat) all of these.
Then input into __Cytoscape__.

In [ ]:
#cat -n louvain_cytoscape_output.csv | tail -1025 | sed -r 's/"//g' - | sed -r 's/,/\t/g' - | sed -r 's/d/\t d/g' - > louvain_t_all

In [1]:
species = ["dmel", "dsim", "dtak", "deug", "dere", "dyak", "dbia", "dele", "dfic", "dtri", "dana"]

In [2]:
spc = [">dmel", ">dsim", ">dtak", ">deug", ">dere", ">dyak", ">dbia", ">dele", ">dfic", ">dtri", ">dana"]

In [3]:
import numpy as np

In [4]:
import sys

In [5]:
import pandas as pd

In [6]:
import os

In [7]:
#presence absence
f = open("louvain_t_all", "r")
z = open("presense_absence_by_line.txt", "w")
r = open("TAD_ranks_chromatin.txt", "w")
array = []
lc = -1
for line in f:
    msa = {}
    l = line.split("\t")
    temp = []
    for i in l:
        s = i.strip()
        temp.append(s[0:4])
    #print(temp)
    for sp in species:
        if sp in temp:
            msa[sp] = 1
        else:
            msa[sp]= 0
    line_array = []
    for k,v in msa.items():  
        line_array.append(v)
        z.write(str(k) + ":" + str(v) + "\t")
    #print(str(dmid) + "\t"+ str(chromatin))
    array.append(line_array)
    #following 7 lines to keep track of TAD rank and chromatin
    lc += 1
    count = line_array.count(1)
    if "dmel" in temp:
        r.write(str(lc) + "\t" + str(count) + "\n")
    else:
        r.write(str(lc) + "\t" + str(count) + "\t" + "no_dmel" + "\n")
    z.write("\n")
z.close()
r.close()

In [39]:
#following 2 cells to turn presence absense matrix to binary fasta

In [8]:
a = np.asarray(array)
b = a.transpose()
df = pd.DataFrame(b)

In [9]:
#for fasta format
df.insert(loc=0, column = 'sp', value=spc)
df.to_csv("boundary_phylo.txt",index=False, sep =' ')

### Use IQtree to create phylogenetic tree from boundary fasta

IQ-TREE v. 2.1.4

In [ ]:
iqtree -s boundary_phylo.fasta -m MFP -pre bounds_mp_const_2 -g suvorov_sub_nv.newick -redo -o dana
#rerun with the model selection from first command
iqtree -s boundary_phylo.fasta -m GTR2+FO -pre bounds_mp_const_2 -g suvorov_sub_nv.newick -redo -o dana
#to run with bootstrapping
iqtree -s boundary_phylo.fasta -m GTR2+FO -pre bounds_mp_const_2 -g suvorov_sub_nv.newick -redo -o dana -bo 100

suvorov_sub_nv.newick

In [ ]:
(dana,(dtri,((((dbia,dtak),(((dere,dyak),(dsim,dmel)),deug)),dele),dfic)));

### Use the following R script to find boundary evolution per mya 

Use this script first to subset the file _schemeA.newick_ from https://github.com/SchriderLab/drosophila_phylogeny/tree/master/tree

In [ ]:
library(ape)
library(treeio)
library(tibble)
library(dplyr)
library(ggtree)
#filter suvorov tree to have only 11 species#sat<-read.newick("schemeA.newick")
subtre<-tree_subset(sat,node="D_melanogaster", levels_back=8)
subtre_td<-as.treedata(subtre)
to_drop<-c("D_parapallidosa","D_parabipectinata","D_bipectinata","D_parabipectinata","D_m_pallens","D_malerkotliana_malerkotliana","D_p_nigrens","D_pseudoananassae_pseudoananassae","D_ercepeae","D_asahinai","D_lacteicornis","D_rufa","D_tani","D_aurauria","D_pectinifera","D_bakoue","D_burlai","D_sp_aff_chauvacae","D_bocqueti","D_nikananu","D_seguyi","D_vulcana","D_jambulina","D_birchii","D_mayri","D_truncata","D_bunnanda","D_serrata","D_punjabiensis","D_watanabei","D_kikkawai","D_leontia","D_kanapiae","D_subpulchrella","D_suzukii","D_orena","D_teissieri_273_3","D_mauritiana","D_sechellia","D_carrolli","D_rhopaloa","D_kurseongensis","D_fuyamai")
subtest<-drop.tip(subtre,to_drop)
write.nexus(subtest, file="suvorov_sub.nexus", translate=TRUE)


In [ ]:
library(ape)
library(treeio)
library(tibble)
library(dplyr)
library(ggtree)

s<-read.newick("suvorov_sub_renamed.newick")
std<-as.treedata(s)
d<-read.newick("RAxML_result.boundary_phylo_constrained")
std<-as.treedata(s)
dtd<-as.treedata(d)
stdt<-as_tibble(std)
dtdt<-as_tibble(dtd)
stdtc<-mutate(stdt,tmrca = branch.length)
dtdtc<-mutate(dtdt,d_subs = branch.length)
stdtct<-as.treedata(stdtc)
dtdtct<-as.treedata(dtdtc)
r<-root(dtdtct, outgroup="dana")
dtd<-as.treedata(r)
dtdt<-as_tibble(dtd)
dtdtc<-mutate(dtdt,d_subs = branch.length)
dtdtct<-as.treedata(dtdtc)
#merge1<-merge_tree(btdtct, dtdtct)
merge<-merge_tree(stdtct, dtdtct)
mta<-as_tibble(merge)
write.jtree(merge)
mta<-mutate(mta,d_subs_per_my=(d_subs*1025)/tmrca)
write.table(mta, file="final_rate_table.txt")